In [8]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [9]:
from Functions_To_Call import plot, lookup_1d, lookup_2d_v2, lookup_2d_v3, integrator,lookup_2d_v4
print(f'Funtions_To_Call Initialized')
from Initial_Parameters import init_volt, init_q_zelle, init_temp, init_soc, kA, cp, m, anzahl_zellen
print(f'Initial_Parameters Initialized')
from Breakpoint_Tables import (soc_steps_ocv, ocv_tbl, temp_steps, R_tbl, SOCsteps, R1_tbl, C1_tbl, R2_tbl, C2_tbl, DeltaOCVdT, SOCsteps_Takano)
print(f'Time_Tables Initialized')

Funtions_To_Call Initialized
Initial_Parameters Initialized
Time_Tables Initialized


In [10]:
Battery_Dataframe = pd.read_csv('BatteryData_0.csv')

In [11]:
Battery_Dataframe['Zeitdifferenz dt [s]'] = 0
Battery_Dataframe['Batterie Leistung [W]'] = 0
Battery_Dataframe['Batterie Strom [A]'] = 0
Battery_Dataframe['Batterie Ladung [C]'] = 0
Battery_Dataframe['SOC [%]'] = init_soc
Battery_Dataframe['R1 [Ohm]'] = 0.0
Battery_Dataframe['R2 [Ohm]'] = 0.0
Battery_Dataframe['C1 [F]'] = 0.0
Battery_Dataframe['C2 [F]'] = 0.0
Battery_Dataframe['OCV [V]'] = 0.0
Battery_Dataframe['U_R1 [V]'] = 0.0
Battery_Dataframe['U_R [V]'] = 0.0
Battery_Dataframe['I_R1 [A]'] = 0.0
Battery_Dataframe['U_R2 [V]'] = 0.0
Battery_Dataframe['I_R2 [A]'] = 0.0
Battery_Dataframe['Q_Irreversibel [W]'] = 0
Battery_Dataframe['Q_Reversibel [W]'] = 0
Battery_Dataframe["Q_Zelle [W]"] = 0
Battery_Dataframe["Q_Batterie [W]"] = 0
Battery_Dataframe['U_Batterie [V]'] = init_volt
Battery_Dataframe['Umgebungs_Temperatur [K]'] = init_temp
Battery_Dataframe['Batterie_Temperatur [K]'] = init_temp

In [12]:
display(Battery_Dataframe)

,Zeit [s],Leistung [kW],Zeitdifferenz dt [s],Batterie Leistung [W],Batterie Strom [A],Batterie Ladung [C],SOC [%],R1 [Ohm],R2 [Ohm],C1 [F],...,I_R1 [A],U_R2 [V],I_R2 [A],Q_Irreversibel [W],Q_Reversibel [W],Q_Zelle [W],Q_Batterie [W],U_Batterie [V],Umgebungs_Temperatur [K],Batterie_Temperatur [K]
0,0.0,0.0,0,0,0,0,50,0.0,0.0,0.0,...,0.0,0.0,0.0,0,0,0,0,46.8,293.15,293.15
1,0.5,0.0,0,0,0,0,50,0.0,0.0,0.0,...,0.0,0.0,0.0,0,0,0,0,46.8,293.15,293.15
2,1.0,0.0,0,0,0,0,50,0.0,0.0,0.0,...,0.0,0.0,0.0,0,0,0,0,46.8,293.15,293.15
3,1.5,0.0,0,0,0,0,50,0.0,0.0,0.0,...,0.0,0.0,0.0,0,0,0,0,46.8,293.15,293.15
4,2.0,0.0,0,0,0,0,50,0.0,0.0,0.0,...,0.0,0.0,0.0,0,0,0,0,46.8,293.15,293.15
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2356,1178.0,0.0,0,0,0,0,50,0.0,0.0,0.0,...,0.0,0.0,0.0,0,0,0,0,46.8,293.15,293.15
2357,1178.5,0.0,0,0,0,0,50,0.0,0.0,0.0,...,0.0,0.0,0.0,0,0,0,0,46.8,293.15,293.15
2358,1179.0,0.0,0,0,0,0,50,0.0,0.0,0.0,...,0.0,0.0,0.0,0,0,0,0,46.8,293.15,293.15
2359,1179.5,0.0,0,0,0,0,50,0.0,0.0,0.0,...,0.0,0.0,0.0,0,0,0,0,46.8,293.15,293.15


In [13]:
U_R1_out = np.zeros_like(Battery_Dataframe["Zeit [s]"].values)
ausgangs_temperatur_1 = init_temp
u_zelle = init_volt
end_time = len(Battery_Dataframe['Zeit [s]'])

In [14]:
for i, row in Battery_Dataframe.iloc[:-1].iterrows():
    # Nebenberechnung um den fortschritt der Berechnung anzuzeigen
    progress = (i / end_time) * 100
    print('\rVerarbeitet: {:.2f}%'.format(progress), end='')

    # Berechnung des Zeitdifferenzials für Integratoren und abspeichern des Wertes in ein Dataframe Eintrag
    Battery_Dataframe.loc[i, 'Zeitdifferenz dt [s]'] = (
        Battery_Dataframe.loc[i + 1, 'Zeit [s]'] - Battery_Dataframe.loc[i, 'Zeit [s]']
    )

    # Zwischenspeichern des Pandas Eintrags in eine Variable für übersichtlicheren Code
    dt = Battery_Dataframe.loc[i, 'Zeitdifferenz dt [s]']

    # Leistung zum Zeitpunkt i aus dem Dataframe (initialisierung in Variable)
    leistung_kw = Battery_Dataframe.at[i, 'Leistung [kW]']  # Ablesen der Leistung aus Dataframe

    # Schritt 1: Umrechnung auf Watt in thermodynamischer konvention (Gain, Multiplikator)
    leistung_w = leistung_kw * -1000                                            # Gain Block oder Multiplikator
    Battery_Dataframe.at[i, 'Batterie Leistung [W]'] = leistung_w               # Einspeisen in das Dataframe

    # Schritt 2: Berechnung des Stroms (Gain, Teiler)
    i_zelle = leistung_w / u_zelle  # Gain Block oder Multiplikator
    Battery_Dataframe.at[i, 'Batterie Strom [A]'] = i_zelle                     # Einspeisen in das Dataframe

    # Schritt 3: Berechnung der Ladung (Integrator)
    q_vorhanden_0 = Battery_Dataframe.at[i, 'Batterie Ladung [C]']              # Ladung aus Zeitpunkt t
    q_vorhanden_1 = q_vorhanden_0 + i_zelle * dt  # Integratorblock

    # Setzen Sie den berechneten Wert für den nächsten Zeitschritt
    Battery_Dataframe.at[i + 1, 'Batterie Ladung [C]'] = q_vorhanden_1          # Einspeisen in das Dataframe

    # Schritt 4: Umrechnung der Ladung in SOC
    divisor_q_cell = 1 / init_q_zelle                                           #(initialisieren der Variable)
    hours_in_seconds = 1 / 3600                                                 #(initialisieren der Variable)
    prozent = 100                                                               #(initialisieren der Variable)
    soc = q_vorhanden_1 * divisor_q_cell * hours_in_seconds * prozent       #Drei Gains oder Multiplikator
    soc += init_soc                                                             #Berücksichtigen des initialwerts
    soc = max(0, min(100, soc))                                                 #Saturation Block
    Battery_Dataframe.at[i + 1, 'SOC [%]'] = soc                                #Einspeisen in das Dataframe

    # Schritt 5: Interpolation der Ohmschen Größen
    # 5.1: 2-D Lookup Interpolation vom OCV [V]
    string_ocv = 'OCV [V]'
    # Aufrufen der Funktion
    interpolated_value = lookup_2d_v4(soc, ausgangs_temperatur_1, soc_steps_ocv, temp_steps, ocv_tbl)
    if i == 0:
        Battery_Dataframe.at[i + 1, string_ocv] = interpolated_value
        Battery_Dataframe.at[i, string_ocv] = interpolated_value
    else:
        Battery_Dataframe.at[i + 1, string_ocv] = interpolated_value


    # 5.2: 2-D Lookup Interpolation von R [Ohm]
    string_R = 'R [Ohm]'
    # Aufrufen der Funktion
    interpolated_value = lookup_2d_v4(soc, ausgangs_temperatur_1, SOCsteps, temp_steps, R_tbl)
    if i == 0:
        Battery_Dataframe.at[i + 1, string_R] = interpolated_value
        Battery_Dataframe.at[i, string_R] = interpolated_value
    else:
        Battery_Dataframe.at[i + 1, string_R] = interpolated_value

    # 5.3: 2-D Lookup Interpolation von R1 [Ohm]
    string_R1 = 'R1 [Ohm]'
    # Aufrufen der Funktion
    interpolated_value = lookup_2d_v4(soc, ausgangs_temperatur_1, SOCsteps, temp_steps, R1_tbl)
    if i == 0:
        Battery_Dataframe.at[i + 1, string_R1] = interpolated_value
        Battery_Dataframe.at[i, string_R1] = interpolated_value
    else:
        Battery_Dataframe.at[i + 1, string_R1] = interpolated_value

    # 5.4: 2-D Lookup Interpolation von R2 [Ohm]
    string_R2 = 'R2 [Ohm]'
    interpolated_value = lookup_2d_v4(soc, ausgangs_temperatur_1, SOCsteps, temp_steps, R2_tbl)
    if i == 0:
        Battery_Dataframe.at[i + 1, string_R2] = interpolated_value
        Battery_Dataframe.at[i, string_R2] = interpolated_value
    else:
        Battery_Dataframe.at[i + 1, string_R2] = interpolated_value

    # 5.5: 2-D Lookup Interpolation von C1 [C]
    string_C1 = 'C1 [F]'
    interpolated_value = lookup_2d_v4(soc, ausgangs_temperatur_1, SOCsteps, temp_steps, C1_tbl)
    if i == 0:
        Battery_Dataframe.at[i + 1, string_C1] = interpolated_value
        Battery_Dataframe.at[i, string_C1] = interpolated_value
    else:
        Battery_Dataframe.at[i + 1, string_C1] = interpolated_value

    # 5.6: 2-D Lookup Interpolation von C2 [C]
    string_C2 = 'C2 [F]'
    interpolated_value = lookup_2d_v4(soc, ausgangs_temperatur_1, SOCsteps, temp_steps, C2_tbl)
    if i == 0:
        Battery_Dataframe.at[i + 1, string_C2] = interpolated_value
        Battery_Dataframe.at[i, string_C2] = interpolated_value
    else:
        Battery_Dataframe.at[i + 1, string_C2] = interpolated_value

    # Schritt 6: Berechnung der Spannung
    # 6.1: Berechnung der Spannung U_R [V]
    r = Battery_Dataframe.at[i + 1, 'R [Ohm]']                                      #(initialisieren der Variable)
    #i_zelle = Battery_Dataframe.at[i, 'Batterie Strom [A]']
    u_r = r * i_zelle                                                               #Gain Block oder Multiplikator
    Battery_Dataframe.at[i + 1, 'U_R [V]'] = u_r                                        #Einspeisen in das Dataframe

    # 6.2: Berechnung von I_R1
    r_1 = Battery_Dataframe.at[i + 1, 'R1 [Ohm]']
    u_r1_prev = Battery_Dataframe.at[i, 'U_R1 [V]']

    # You need to ensure i_zelle is defined here, maybe it's a constant or should be fetched from the DataFrame
    i_zelle = Battery_Dataframe.at[i, 'Batterie Strom [A]'] # If it's a column

    # Calculate I_R1
    i_r1 = u_r1_prev / r_1
    Battery_Dataframe.at[i, 'I_R1 [A]'] = i_r1

    # Calculate Delta_I_R1
    delta_i_1 = i_zelle - i_r1
    Battery_Dataframe.at[i, 'Delta_I_R1 [A]'] = delta_i_1

    # Calculate dU_R1/dt
    c_1 = Battery_Dataframe.at[i + 1, 'C1 [F]']  # Assuming C1 is in farads and not coulombs, as that's the SI unit for capacitance
    differenzial_u_r1 = delta_i_1 / c_1

    # Calculate U_R1 through Integrator
    u_r1 = u_r1_prev + differenzial_u_r1 * dt
    Battery_Dataframe.at[i + 1, 'U_R1 [V]'] = u_r1

    # 6.5: Berechnung von I_R2
    r_2 = Battery_Dataframe.at[i + 1, 'R2 [Ohm]']  # (initialisieren der Variable)
    u_r2_prev = Battery_Dataframe.at[i, 'U_R2 [V]']  # (initialisieren der Variable)

    # You need to ensure i_zelle is defined here, maybe it's a constant or should be fetched from the DataFrame
    i_zelle = Battery_Dataframe.at[i, 'Batterie Strom [A]'] # If it's a column

    # Calculate I_R2
    i_r2 = u_r2_prev / r_2
    Battery_Dataframe.at[i, 'I_R2 [A]'] = i_r2

    # Calculate Delta_I_R2
    delta_i_2 = i_zelle - i_r2
    Battery_Dataframe.at[i, 'Delta_I_R2 [A]'] = delta_i_2

    # Calculate dU_R2/dt
    c_2 = Battery_Dataframe.at[i + 1, 'C2 [F]']  # Assuming C2 is in farads
    differenzial_u_r2 = delta_i_2 / c_2

    # Calculate U_R2 through Integrator
    u_r2 = u_r2_prev + differenzial_u_r2 * dt
    Battery_Dataframe.at[i + 1, 'U_R2 [V]'] = u_r2

    # 6.9 Berechnung der gesamten Zellspannung
    u_zelle = (
            Battery_Dataframe.at[i + 1, 'U_R1 [V]'] +
            Battery_Dataframe.at[i + 1, 'U_R2 [V]'] +
            Battery_Dataframe.at[i + 1, 'U_R [V]'] +
            Battery_Dataframe.at[i + 1, 'OCV [V]']
    ) * anzahl_zellen                                                           #Add Block

    Battery_Dataframe.at[i + 1, 'U_Batterie [V]'] = u_zelle                              #Einspeisen in das Dataframe

    # Schritt 7: Berechnung der irreversiblen Wärme
    # 7.1: Berechnung der irreversiblen Wärme der ohmschen Größen
    q_r = r * (i_zelle ** 2)  # Gain Block oder Multiplikator
    q_r1 = r_1 * (i_r1 ** 2)  # Gain Block oder Multiplikator
    q_r2 = r_2 * (i_r2 ** 2)  # Gain Block oder Multiplikator

    # 7.2: Berechnung der irreversiblen Wärme
    q_irreversibel = q_r + q_r1 + q_r2  # Add Block
    Battery_Dataframe.at[i + 1, 'Q_Irreversibel [W]'] = q_irreversibel  # Einspeisen in das Dataframe

    # Schritt 8: Berechnung von reversibler Wärme
    delta_ocv = lookup_1d(soc, SOCsteps_Takano, DeltaOCVdT)  # Nutzung von 1-D Lookup Table
    ist_temperatur = Battery_Dataframe.at[i + 1, 'Umgebungs_Temperatur [K]']  # Auslesen der ist-Temperatur
    q_reversibel = ist_temperatur * i_zelle * delta_ocv  # Gain Block oder Multiplikator
    Battery_Dataframe.at[i + 1, "Q_Reversibel [W]"] = q_reversibel  # Einspeisen in das Dataframe

    # Schritt 9: Berechnung von Zellwärme als Summe und einzeln
    q_zelle = q_reversibel + q_irreversibel  # Add Block
    Battery_Dataframe.at[i + 1, "Q_Zelle [W]"] = q_zelle  # Einspeisen in das Dataframe

    q_zelle_summe = anzahl_zellen * q_zelle  # Gain Block oder Multiplikator
    Battery_Dataframe.at[i + 1, "Q_Batterie [W]"] = q_zelle_summe  # Einspeisen in das Dataframe

    # Schritt 10: Berechnung der Temperatur (Siehe Abschnitt Schmidt Bat Lab. Aufgabenstellung)
    delta_temperatur = ist_temperatur - ausgangs_temperatur_1  # Substract Block
    q_kuehlung_negativ = kA * delta_temperatur  # Negativ thermodynamische konvention
    sum_q = q_kuehlung_negativ + q_zelle  # Summe der Wärme
    d_temperatur = sum_q / (m * cp)  # Berechnung des temperatur differenzials
    ausgangs_temperatur_0 = Battery_Dataframe.at[i , 'Batterie_Temperatur [K]']  # Temperatur aus letzter iteration
    ausgangs_temperatur_1 = ausgangs_temperatur_0 + d_temperatur * dt  # Integrator Block

    # Store this "output" temperature
    Battery_Dataframe.at[i + 1, 'Batterie_Temperatur [K]'] = ausgangs_temperatur_1  # Einspeisen in das Dataframe

print('\nSimulation abgeschlossen!')

Verarbeitet: 70.90%
Warnung 2D: Der SOC-Wert von 297.16853675843777 liegt außerhalb der Grenzen und muss extrapoliert werden.

Warnung 2D: Der SOC-Wert von 297.16853675843777 liegt außerhalb der Grenzen und muss extrapoliert werden.

Warnung 2D: Der SOC-Wert von 297.16853675843777 liegt außerhalb der Grenzen und muss extrapoliert werden.

Warnung 2D: Der SOC-Wert von 297.16853675843777 liegt außerhalb der Grenzen und muss extrapoliert werden.

Warnung 2D: Der SOC-Wert von 297.16853675843777 liegt außerhalb der Grenzen und muss extrapoliert werden.
Verarbeitet: 70.94%
Warnung 2D: Der SOC-Wert von 297.20057362928947 liegt außerhalb der Grenzen und muss extrapoliert werden.

Warnung 2D: Der SOC-Wert von 297.20057362928947 liegt außerhalb der Grenzen und muss extrapoliert werden.

Warnung 2D: Der SOC-Wert von 297.20057362928947 liegt außerhalb der Grenzen und muss extrapoliert werden.

Warnung 2D: Der SOC-Wert von 297.20057362928947 liegt außerhalb der Grenzen und muss extrapoliert werden.

In [15]:
display(Battery_Dataframe)

,Zeit [s],Leistung [kW],Zeitdifferenz dt [s],Batterie Leistung [W],Batterie Strom [A],Batterie Ladung [C],SOC [%],R1 [Ohm],R2 [Ohm],C1 [F],...,Q_Irreversibel [W],Q_Reversibel [W],Q_Zelle [W],Q_Batterie [W],U_Batterie [V],Umgebungs_Temperatur [K],Batterie_Temperatur [K],R [Ohm],Delta_I_R1 [A],Delta_I_R2 [A]
0,0.0,0.0,0.5,0,0.0,0.000000,50.000000,0.001338,0.001301,8609.705022,...,0.000000,0.0,0.000000,0.000000,46.800000,293.15,293.150000,0.002195,0.000000,0.000000
1,0.5,0.0,0.5,0,0.0,0.000000,50.000000,0.001338,0.001301,8609.705022,...,0.000000,0.0,0.000000,0.000000,48.107800,293.15,293.150000,0.002195,0.000000,0.000000
2,1.0,0.0,0.5,0,0.0,0.000000,50.000000,0.001338,0.001301,8609.705022,...,0.000000,0.0,0.000000,0.000000,48.107800,293.15,293.150000,0.002195,0.000000,0.000000
3,1.5,0.0,0.5,0,0.0,0.000000,50.000000,0.001338,0.001301,8609.705022,...,0.000000,0.0,0.000000,0.000000,48.107800,293.15,293.150000,0.002195,0.000000,0.000000
4,2.0,0.0,0.5,0,0.0,0.000000,50.000000,0.001338,0.001301,8609.705022,...,0.000000,0.0,0.000000,0.000000,48.107800,293.15,293.150000,0.002195,0.000000,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2356,1178.0,0.0,0.5,0,-0.0,-731.248057,46.372777,0.001028,0.001072,8250.345092,...,1.872388,-0.0,1.872388,24.341049,48.535820,293.15,307.036086,0.001499,-15.225099,-37.225061
2357,1178.5,0.0,0.5,0,-0.0,-731.248057,46.372777,0.001031,0.001072,8261.907623,...,1.724250,-0.0,1.724250,22.415253,48.504529,293.15,306.980471,0.001500,-14.292725,-35.846411
2358,1179.0,0.0,0.5,0,-0.0,-731.248057,46.372777,0.001034,0.001072,8273.509627,...,1.588187,-0.0,1.588187,20.646425,48.474671,293.15,306.924703,0.001502,-13.420866,-34.516273
2359,1179.5,0.0,0.5,0,-0.0,-731.248057,46.372777,0.001036,0.001071,8285.143599,...,1.463155,-0.0,1.463155,19.021011,48.446173,293.15,306.868814,0.001504,-12.605406,-33.233019


In [16]:
columns_to_keep = ['U_Batterie [V]', 'Zeit [s]']
Battery_Dataframe = Battery_Dataframe[columns_to_keep]

In [17]:
Battery_Dataframe.to_csv('Gesamtspannung_Python_full.csv', index=False)